In [1]:
pip install comtypes


Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pycaw

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install keyboard

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install --upgrade pycaw

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# volume_hotkeys.py
import sys
import time

# Windows-only check
if sys.platform != "win32":
    print("This script works only on Windows (pycaw uses Windows Core Audio).")
    sys.exit(1)

from comtypes import CLSCTX_ALL, cast
from ctypes import POINTER
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume
import keyboard  # keyboard needs admin privileges for global hotkeys on some systems

# --- Setup audio interface ---
speakers = AudioUtilities.GetSpeakers()
volume = speakers.EndpointVolume

# Helper wrappers
def get_volume():
    """Return master volume as float 0.0-1.0"""
    try:
        return volume.GetMasterVolumeLevelScalar()
    except Exception as e:
        print("Error reading volume:", e)
        return None

def set_volume_scalar(scalar):
    """Set master volume (0.0 - 1.0)"""
    scalar = max(0.0, min(1.0, float(scalar)))
    volume.SetMasterVolumeLevelScalar(scalar, None)
    print(f"Volume set to {int(scalar*100)}%")

def change_volume(delta):
    """Change volume by delta (float, e.g. 0.05 = 5%)"""
    cur = get_volume()
    if cur is None:
        return
    set_volume_scalar(cur + delta)

def toggle_mute():
    """Toggle mute state"""
    muted = volume.GetMute()
    volume.SetMute(0 if muted else 1, None)
    print("Muted" if not muted else "Unmuted")

def print_status():
    v = get_volume()
    muted = bool(volume.GetMute())
    print(f"Volume: {int(v*100)}%  Muted: {muted}")

# --- Hotkey bindings ---
STEP = 0.05  # 5% step

print("Volume hotkeys active:")
print("  Ctrl+Alt+Up    -> Volume up  (+5%)")
print("  Ctrl+Alt+Down  -> Volume down (-5%)")
print("  Ctrl+Alt+M     -> Toggle mute")
print("  Ctrl+Alt+S     -> Show status")
print("  Ctrl+Alt+Q     -> Quit")

keyboard.add_hotkey('ctrl+alt+up', lambda: change_volume(STEP))
keyboard.add_hotkey('ctrl+alt+down', lambda: change_volume(-STEP))
keyboard.add_hotkey('ctrl+alt+m', toggle_mute)
keyboard.add_hotkey('ctrl+alt+s', print_status)

# Quit handler
def quit_program():
    print("Quitting...")
    keyboard.unhook_all_hotkeys()
    sys.exit(0)

keyboard.add_hotkey('ctrl+alt+q', quit_program)

# Keep the script alive
try:
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    quit_program()


Volume hotkeys active:
  Ctrl+Alt+Up    -> Volume up  (+5%)
  Ctrl+Alt+Down  -> Volume down (-5%)
  Ctrl+Alt+M     -> Toggle mute
  Ctrl+Alt+S     -> Show status
  Ctrl+Alt+Q     -> Quit
Volume set to 71%
Volume set to 76%
Volume set to 71%
Volume set to 66%
Volume set to 71%
Volume set to 76%
Volume set to 81%
Volume set to 76%
Volume set to 71%
Volume set to 66%
Muted
Unmuted
Volume: 66%  Muted: False
Quitting...
